In [36]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

from nltk.tokenize import word_tokenize

nltk.download('wordnet')
from nltk.corpus import wordnet as wn

from nltk.probability import FreqDist

import spacy
!python -m spacy download en
sp = spacy.load("en_core_web_sm")

from IPython.display import display, HTML

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 39.4 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [37]:
output_df = pd.read_excel("output.xls")
text_df = pd.read_csv("18_093_attachment_text_Aug21.csv", encoding="ISO-8859-1")[["Date", "Attachment_text", "cn_doc_id", "Summary", "BRCID"]]

/tmp/ipykernel_584802/2352958704.py:2: DtypeWarning: Columns (3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,26

In [38]:
complete_data = pd.merge(output_df, text_df, left_on='CN_Doc_ID', right_on='cn_doc_id', how='right')
complete_data.sort_values("ethnicitycleaned", axis=0, inplace=True)
complete_data.rename(columns={"ethnicitycleaned": "Ethnicity", "Attachment_text": "Patient_Summary", "age": "Age", "Gender_ID": "Gender", "diagnosis_date": "Diagnosis_Date", "document_date": "Document_Date", "CN_Doc_ID": "unique_id"}, inplace=True)
complete_data = complete_data.dropna(subset=["Ethnicity", "Patient_Summary"])
print("MERGED COMPLETE DATA")

complete_data['Patient_Summary'] = [sp(complete_data['Patient_Summary'].iloc[i]) for i in range(complete_data.shape[0])]
complete_data['tokens'] = [[word.text for word in complete_data['Patient_Summary'].iloc[i]] for i in range(complete_data.shape[0])]
print("GET TOKENS FROM COMPLETE DATA")

complete_data['no_stop'] = ''
for i in range(complete_data.shape[0]):
    complete_data['no_stop'].iloc[i] = [t for t in complete_data['tokens'].iloc[i] if t.lower() not in stop_words and t.isalnum() and not t.isnumeric() and t.lower() not in ["i", "is", "be", "am", "are", "the", "SLaM", "" "i'm", "i’m", "zzzzz"]]
print("GET NON-STOPWORD TOKENS FROM COMPLETE DATA")

MERGED COMPLETE DATA
GET TOKENS FROM COMPLETE DATA
GET NON-STOPWORD TOKENS FROM COMPLETE DATA


In [39]:
for i in range(complete_data.shape[0]):
    full_corpus += ' '.join(word for word in complete_data['no_stop'].iloc[i])

print(len(full_corpus))

9536292


In [ ]:
print(full_corpus[:10000])

In [40]:
toks = word_tokenize(full_corpus)
fdist = FreqDist(word.lower() for word in toks)
top_thirty = fdist.most_common(200)
print(top_thirty)

[('team', 9501), ('dr', 8956), ('please', 8481), ('health', 8405), ('mental', 7902), ('assessment', 7608), ('service', 7343), ('appointment', 7090), ('would', 6449), ('contact', 6154), ('qqqqq', 5922), ('time', 5747), ('also', 5704), ('gp', 5700), ('care', 5520), ('road', 5360), ('mood', 5172), ('support', 5010), ('hospital', 4964), ('date', 4712), ('referral', 4389), ('medication', 4182), ('family', 4161), ('dear', 4081), ('number', 4041), ('ms', 3986), ('history', 3903), ('risk', 3801), ('sincerely', 3641), ('low', 3474), ('current', 3462), ('self', 3456), ('plan', 3411), ('mg', 3381), ('psychological', 3379), ('information', 3357), ('mr', 3345), ('reported', 3339), ('may', 3319), ('thoughts', 3315), ('treatment', 3283), ('telephone', 3277), ('centre', 3272), ('services', 3246), ('said', 3243), ('mother', 3156), ('home', 3078), ('social', 3072), ('perinatal', 3069), ('children', 3024), ('clinical', 3016), ('us', 2968), ('help', 2948), ('group', 2938), ('anxiety', 2931), ('tel', 2931)

In [46]:
list = ['aware', 'clinician', 'doctor', 'childcare', 'alcohol', 'risk', 'personality', 'community', 'difficult', 'team', 'health', 'assessment', 'appointment', 'medication', 'road', 'family', 'sincerely', 'history', 'mood', 'hospital', 'psychological', 'perinatal', 'information', 'thoughts', 'treatment', 'social', 'anxious', 'services', 'telephone', 'confidential', 'clinical', 'symptoms', 'feeling', 'crisis', 'well', 'diagnosis', 'father']
synonyms = {}
for word in list:
    synonyms[word] = []
    for entry in wn.synsets(word):
        for lemma in entry.lemmas():
            synonyms[word].append(lemma.name())
for key in synonyms.keys():
    curr_list = synonyms[key]
    for e in curr_list:
        if e not in toks:
            print(f"removing {e}")
            synonyms[key].remove(e)
for entry in synonyms.keys():
    print(f"KEY: {entry}\tSYNONYMS: {synonyms[entry]}\n")

removing cognizant
removing Dr.
removing Doctor_of_the_Church
removing Dr.
removing doctor_up
removing mend
removing bushel
removing furbish_up
removing touch_on
removing child_care
removing alcoholic_drink
removing intoxicant
removing peril
removing endangerment
removing peril
removing risk_of_infection
removing risk_of_exposure
removing put_on_the_line
removing gamble
removing take_chances
removing run_a_risk
removing community_of_interests
removing residential_area
removing biotic_community
removing squad
removing team_up
removing wellness
removing judgment
removing designation
removing appointee
removing medicament
removing menage
removing family_unit
removing family_line
removing kinfolk
removing sept
removing kinsperson
removing syndicate
removing mob
removing fellowship
removing unfeignedly
removing sincerely_yours
removing chronicle
removing humor
removing climate
removing infirmary
removing selective_information
removing thought_process
removing intellection
removing sentiment